# Representation Learning


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tarfile
import glob
from datetime import datetime
import math
import csv

In [17]:
# Load and inspect imputed data from set A (training)
df = pd.read_parquet('imputed-data/imputed-set-a.parquet')
df = df.drop('ICUType', axis=1) # we shouldn't use ICUType
df['RecordID'] = df['RecordID'].astype(pd.Int64Dtype())
df_labels = pd.read_parquet('processed-data/processed-outcomes-a.parquet')

In [18]:
df.head()

,Time,Age,BUN,Creatinine,GCS,Gender,Glucose,HCO3,HCT,HR,...,FiO2,Lactate,MAP,MechVent,PaCO2,PaO2,SaO2,SysABP,TroponinI,pH
0,00:00,35.0,17.5,1.05,15.0,0.0,136.5,22.0,37.45,92.5,...,1.0,2.500,83.0,1.0,39.0,248.5,NaN,121.000000,NaN,7.34
1,01:00,35.0,20.0,1.00,14.0,0.0,142.0,23.0,32.50,88.5,...,1.0,2.350,84.0,1.0,41.0,261.5,97.0,121.000000,3.1,7.37
2,02:00,35.0,19.0,0.90,15.0,0.0,140.0,23.0,31.60,112.0,...,0.7,2.325,81.0,1.0,41.0,236.0,98.0,117.500000,4.2,7.37
3,03:00,35.0,68.0,2.30,15.0,0.0,603.0,11.0,25.50,113.0,...,0.6,2.300,80.0,1.0,41.0,193.5,98.0,116.666667,1.1,7.36
4,04:00,35.0,68.0,2.30,15.0,0.0,603.0,11.0,25.50,112.0,...,0.5,2.300,78.5,1.0,41.0,183.0,98.0,115.875000,1.4,7.36


### Q3.1 Pretraining and Linear Probes (4 pts)

### Q3.2 Simulate label scarcity (4 pts)

### Q3.3: Visualising Learned Representations (4 Pts)